In [1]:
import open3d
import numpy as np
import os
import glob
import tqdm
import copy

import utils.registration as registration
import utils.functions as functions
import utils.transform as transform
import utils.pointcloud as pointcloud
import utils.fread as fread
import utils.FCGF as FCGF

from utils.config import Config
from PIL import Image

In [2]:
from dataclasses import dataclass
import json

@dataclass
class DepthCameraParams:
    fx: float
    fy: float
    px: float
    py: float
    width: int
    height: int
    depth_scale: float
    intrinsics: open3d.camera.PinholeCameraIntrinsic
    
    def __init__(self, metadata_fname: str):
        with open(metadata_fname, "r") as f:
            metadata = json.load(f)
            self.fx = metadata["fx"]
            self.fy = metadata["fy"]
            self.px = metadata["px"]
            self.py = metadata["py"]
            self.width = metadata["width"]
            self.height = metadata["height"]
            self.depth_scale = metadata["depth_scale"]
            self.intrinsics = open3d.camera.PinholeCameraIntrinsic(self.width, self.height, self.fx, self.fy, self.px, self.py)
    

In [3]:
def depth_image_to_point_cloud(camera_params: DepthCameraParams, depth_image_fname: str):
    fx, fy = camera_params.intrinsics.get_focal_length()
    cx, cy = camera_params.intrinsics.get_principal_point()
    
    depth_image = Image.open(depth_image_fname).convert("I")
    
    z = np.array(depth_image) / camera_params.depth_scale

    x, y = np.meshgrid(np.arange(0, z.shape[1]), np.arange(0, z.shape[0]))
    x = (x - cx) * z / fx
    y = (y - cy) * z / fy

    xyz = np.stack([x, y, z], axis=2)
    xyz = xyz[z > 0]
    xyz = np.reshape(xyz, (-1, 3))
    
    xpcd = open3d.geometry.PointCloud()
    xpcd.points = open3d.utility.Vector3dVector(xyz)
    
    return xpcd

In [4]:
def compute_fpfh(pcd: open3d.geometry.PointCloud, voxel_size: float):
    radius_normal = voxel_size * 2
    pcd.estimate_normals(open3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    pcd_fpfh = open3d.pipelines.registration.compute_fpfh_feature(
        pcd, open3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    
    return pcd, pcd_fpfh

In [5]:
def exec_ransac(source, target, source_feat, target_feat, n_ransac, threshold):
    return open3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source, target, source_feat, target_feat, True, threshold,
        open3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        n_ransac, [
            open3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            open3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(threshold)
        ],
        open3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 0.999))
    

def exec_icp(source, target, threshold, trans_init, max_iteration=30):
    return open3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        open3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        open3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=max_iteration)
    )


In [6]:
camera_params = DepthCameraParams("../liloc/data/raw_data/exp_9/metadata/device-3-aligned.json")

In [47]:
config = Config(
    sequence_dir="../liloc/data/raw_data",
    feature_dir="../liloc/data/features",
    output_dir="temp",
    experiment="exp_9",
    trial="trial_1",
    subject="subject-1",
    sequence="02",
    groundtruth_dir="data/trajectories/groundtruth",
    voxel_size=0.05,
    target_fps=20
)

In [48]:
sequence_ts = fread.get_timstamps_from_images(config.get_sequence_dir(), ext=".depth.png")
sequence_ts = fread.sample_timestamps(sequence_ts, config.target_fps)

elapsed_ts = sequence_ts - sequence_ts[0]
startt_idx = np.argwhere(elapsed_ts >= 4000)[0][0]
sequence_ts = sequence_ts[startt_idx:]

num_frames = len(sequence_ts)
print(f"-- Number of frames: {num_frames}")

-- Number of frames: 356


In [49]:
local_pcds = []
local_fpfh = []

for t in tqdm.trange(num_frames):
    depth_image_fname = os.path.join(config.get_sequence_dir(), f"frame-{sequence_ts[t]}.depth.png")
    pcd = depth_image_to_point_cloud(camera_params, depth_image_fname)
    pcd = open3d.voxel_down_sample(pcd, config.voxel_size)
    pcd, fpfh = registration.compute_fpfh(pcd, config.voxel_size, False)
    
    local_pcds.append(pcd)
    local_fpfh.append(fpfh)

100%|██████████| 356/356 [00:17<00:00, 20.16it/s]


In [50]:
local_t = [np.identity(4)]

for t in tqdm.trange(num_frames - 1):
    source, source_fpfh = copy.deepcopy(local_pcds[t + 1]), local_fpfh[t + 1]
    target, target_fpfh = copy.deepcopy(local_pcds[t]), local_fpfh[t]

    reg_result = registration.exec_ransac(source, target, source_fpfh, target_fpfh, n_ransac=4, threshold=0.05)
    reg_result = registration.exec_icp(source, target, threshold=0.05, trans_init=reg_result.transformation, max_iteration=200)

    local_t.append(reg_result.transformation)

100%|██████████| 355/355 [04:54<00:00,  1.21it/s]


In [51]:
trajectory_t = [np.identity(4)]

for t in tqdm.trange(1, num_frames):
    trajectory_t.append(np.dot(trajectory_t[t - 1], local_t[t]))

100%|██████████| 355/355 [00:00<00:00, 118323.10it/s]


In [52]:
trajectory = []

for i in range(num_frames):
    x = copy.deepcopy(local_pcds[i])
    x.transform(trajectory_t[i])
    trajectory.append(x)
    
trajectory = pointcloud.merge_pcds(trajectory, config.voxel_size)

In [53]:
open3d.visualization.draw_geometries([trajectory])

In [37]:
for i in range(50, 60):
    source = copy.deepcopy(local_pcds[i + 1])
    target = copy.deepcopy(local_pcds[i])
    
    source.transform(local_t[i + 1])
    
    registration.view(source, target, local_t[i + 1])